In [3]:
import torch

# Verificar si CUDA está disponible
print("CUDA disponible:", torch.cuda.is_available())

# Si hay GPU, mostrar nombre
if torch.cuda.is_available():
    print("Nombre de GPU:", torch.cuda.get_device_name(0))
    print("Número de GPUs:", torch.cuda.device_count())
else:
    print("No se detectó GPU. Usando CPU.")

CUDA disponible: True
Nombre de GPU: Tesla T4
Número de GPUs: 1


In [ ]:
import os
import sys
from google.colab import drive

# Verificar si Drive ya está montado
if os.path.exists('/content/drive/MyDrive'):
    print("Google Drive ya está montado")
else:
    print("Montando Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=False)
        print("Drive montado exitosamente")
    except Exception as e:
        print(f"Error al montar Drive: {e}")
        print("\nSoluciones:")
        print("1. Runtime > Restart runtime (Ctrl+M+.)")
        print("2. Ejecuta esta celda de nuevo")
        print("3. Verifica tu conexión a internet")
        print("4. Revisa permisos de tu cuenta de Google")
        raise

# Ruta al proyecto en Drive (ajusta si está en otra ubicación)
project_path = '/content/drive/MyDrive/dlcv25-assignment-2-Cesar421'

# Si no existe en la raíz, buscar en Colab Notebooks
if not os.path.exists(project_path):
    alternative_path = '/content/drive/MyDrive/Colab Notebooks/dlcv25-assignment-2-Cesar421'
    if os.path.exists(alternative_path):
        project_path = alternative_path
        print(f"Proyecto encontrado en: Colab Notebooks/")

# Verificar que el proyecto existe
if os.path.exists(project_path):
    os.chdir(project_path)
    print(f"Proyecto cargado desde: {os.getcwd()}")
    
    # Agregar src al PYTHONPATH
    sys.path.insert(0, os.path.join(project_path, 'src'))
    print(f"PYTHONPATH configurado")
    
    # Mostrar estructura del proyecto
    print("\nEstructura del proyecto:")
    !ls -lh
else:
    print(f"ERROR: No se encontró el proyecto en {project_path}")
    print("\nOpciones:")
    print("1. Verifica que subiste la carpeta 'dlcv25-assignment-2-Cesar421' a tu Google Drive")
    print("2. Busca en qué carpeta está:")
    print("\nContenido de tu Drive:")
    !find /content/drive/MyDrive -maxdepth 2 -type d -name "*dlcv*" 2>/dev/null
    print("\nCarpetas principales:")
    !ls -d /content/drive/MyDrive/*/ | head -10

# Entrenamiento DLCV con GPU de Google Colab

Este notebook ejecuta el proyecto `dlcv25-assignment-2-Cesar421` usando GPU de Google Colab.

**Pasos:**
1. Celda 1: Verificar GPU disponible
2. Celda 2: Montar Google Drive y cargar proyecto
3. Celda 3: Instalar dependencias
4. Celda 4: Ejecutar entrenamiento

In [ ]:
# Instalar dependencias del proyecto
print("Instalando dependencias...")

# Instalar el paquete en modo editable
!pip install -e . -q

print("Dependencias instaladas correctamente")

# Verificar instalación
print("\nVerificando instalación:")
!pip show torch torchvision matplotlib

In [ ]:
# Configuración 1: UNDERFITTING (Modelo muy simple)
print("Iniciando entrenamiento - UNDERFITTING")
print("=" * 60)

!python tools/train.py --run_name underfit --epochs 10 --conv_layers 2 --filters_conv1 8 --filters_conv2 16 --dense_units 32 --batch_size 64 --base_lr 0.001 --data_root ./data

print("\nEntrenamiento UNDERFIT completado")
print("Resultados guardados en: results/underfit.csv")
print("Modelo guardado en: saved_models/underfit.pth")

In [ ]:
# Configuración 2: GOOD FIT (Modelo balanceado con augmentation)
print("Iniciando entrenamiento - GOOD FIT")
print("=" * 60)

!python tools/train.py --run_name goodfit --epochs 50 --conv_layers 3 --filters_conv1 32 --filters_conv2 64 --filters_conv3 128 --dense_units 256 --batch_size 64 --base_lr 0.001 --horizontal_flip_prob 0.5 --rotation_degrees 15 --do_early_stopping --data_root ./data

print("\nEntrenamiento GOODFIT completado")
print("Resultados guardados en: results/goodfit.csv")
print("Modelo guardado en: saved_models/goodfit.pth")

In [ ]:
# Configuración 3: OVERFITTING (Modelo grande con pocos datos)
print("Iniciando entrenamiento - OVERFITTING")
print("=" * 60)

!python tools/train.py --run_name overfit --epochs 100 --conv_layers 3 --filters_conv1 64 --filters_conv2 256 --filters_conv3 512 --dense_units 1024 --batch_size 32 --base_lr 0.001 --subset_size 500 --data_root ./data
print("\nEntrenamiento OVERFIT completado")
print("Resultados guardados en: results/overfit.csv")
print("Modelo guardado en: saved_models/overfit.pth")

In [ ]:
# Generar plots de comparación
print("Generando plots de comparación...")

!python tools/plot.py results/underfit.csv results/goodfit.csv results/overfit.csv

print("\nPlot de comparación generado: results/comparison_plot.png")

In [ ]:
# Visualizar mejor modelo (goodfit)
print("Generando visualizaciones del mejor modelo...")

!python tools/visualize.py \
    saved_models/goodfit.pth \
    --conv_layers 3 \
    --filters_conv1 32 \
    --filters_conv2 64 \
    --filters_conv3 128 \
    --dense_units 256 \
    --data_root ./data

print("\nVisualizaciones generadas:")
print("   - results/sample_predictions.png")
print("   - results/confusion_matrix.png")

In [ ]:
# Descargar resultados a tu PC
from google.colab import files
import zipfile

print("Empaquetando resultados...")

# Crear ZIP con resultados
!zip -r results_and_models.zip results/ saved_models/ -q

print("Archivo ZIP creado")
print("\nDescargando resultados...")

# Descargar el ZIP
files.download('results_and_models.zip')

print("\n¡Listo! Los resultados se descargaron a tu PC")
print("\nContenido del ZIP:")
print("   - results/*.csv (métricas de entrenamiento)")
print("   - results/*.png (plots y visualizaciones)")
print("   - saved_models/*.pth (modelos entrenados)")

In [1]:
import torch

# Cargar el archivo
checkpoint = torch.load('fixture_weights_dont_delete_this.pth', map_location='cpu')

# Ver las claves (nombres de capas)
print("Capas en el modelo:")
for key in checkpoint.keys():
    print(f"  {key}: {checkpoint[key].shape}")

: 